In [109]:
from azureml.core import Workspace
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from azureml.core import Experiment
import time

import numpy as np

from azureml.core import ScriptRunConfig

In [28]:
ws = Workspace.create("breadCancerPca2", subscription_id="030117c9-e67b-4755-af85-4683fe8eddf2", \
                       resource_group="dp100_cc", location='francecentral')

Deploying AppInsights with name breadcaninsightsa1eda953.
Deployed AppInsights with name breadcaninsightsa1eda953. Took 3.18 seconds.
Deploying KeyVault with name breadcankeyvault80bb5840.
Deploying StorageAccount with name breadcanstorage4aed465b4.
Deployed KeyVault with name breadcankeyvault80bb5840. Took 17.42 seconds.
Deployed StorageAccount with name breadcanstorage4aed465b4. Took 22.73 seconds.
Deploying Workspace with name breadCancerPca2.
Deployed Workspace with name breadCancerPca2. Took 19.83 seconds.


In [7]:
data = load_breast_cancer()
type(data)

sklearn.utils.Bunch

In [8]:
data.data

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [9]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [106]:
print(np.shape(X))
print(np.shape(y))

(569, 30)
(569,)


In [29]:
experiment = Experiment(ws, "experiment")

In [30]:
Experiment.list(ws)

[Experiment(Name: experiment,
 Workspace: breadCancerPca2)]

In [72]:
experiment.set_tags({"tag1":"classification"})
experiment.set_tags({"tag2":"Breast Cancer"})
experiment.set_tags({"tag3":"PCA"})

In [102]:
def train_model(X, y, run, component):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    # Make an instance of the Model
    pca = PCA(component)

    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)

    model = LogisticRegression()
    print(f'training: component number {component}')
    start_time = time.time()
    model.fit(X_train, y_train)
    duration = time.time() - start_time
    print('done')
    run.log("Number of component", component_number)
    run.log("Run time Duration", duration)
    run.log("Accuracy", model.score(X_test,y_test))
    run.complete()
    
    return model

In [105]:
run = experiment.start_logging(outputs=None, snapshot_directory=False)
X, y = data.data, data.target

for component_number in (np.arange(1, 8, 1)):
    model= train_model(X, y, run, component_number)

training: component number 1
done
training: component number 2
done
training: component number 3
done
training: component number 4
done
training: component number 5
done
training: component number 6
done
training: component number 7
done
training: component number 8
done


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
